In [233]:
import pandas as pd
import urllib.request
import os
from datetime import datetime

In [234]:
# Функція для перевірки, чи файл вже існує
def file_exists(province_id, save_dir):
    # Шукаємо файли, які містять індекс області у назві
    for filename in os.listdir(save_dir):
        if f"vhi_province_{province_id}" in filename:
            return os.path.join(save_dir, filename)
    return None


In [235]:
# Функція для завантаження файлів 
def download_data(id, save_dir="vhi_data"):
    # Створення директорії для зберігання файлів, якщо вона ще не існує
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    for i in range(1, id + 1):
        url = f'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={i}&year1=1981&year2=2024&type=Mean'
        
        # Формування імені файлу з датою і часом
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f'vhi_province_{i}_{timestamp}.csv'
        filepath = os.path.join(save_dir, filename)

        # Перевіряємо, чи файл для даної області вже існує
        existing_file = file_exists(i, save_dir)
        if existing_file:
            continue  # Пропускаємо завантаження, якщо файл існує

        # Завантажуємо файл і зберігаємо його
        vhi_url = urllib.request.urlopen(url)
        with open(filepath, 'wb') as out:
            out.write(vhi_url.read())


In [236]:
# Функція для видалення конкретних HTML-тегів вручну
def clean_html_tags(text):
    clean_text = text.replace('<tt>', '').replace('</tt>', '').replace('<pre>', '').replace('</pre>', '')
    return clean_text

def load_files_to_dataframe(directory_path):
    # Створюємо порожній список для збереження DataFrame з кожного файлу
    df_list = []
    i = 0
    
    # Проходимо по всіх файлах у директорії
    for filename in os.listdir(directory_path):
        if filename.endswith(".csv"):  # Обробляємо тільки файли CSV
            file_path = os.path.join(directory_path, filename)
            i += 1
            # Зчитуємо CSV файл у DataFrame, пропускаючи перші 3 рядки з метаданими
            headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
            df = pd.read_csv(file_path, header=1, names=headers)
            
            # Очищуємо всі стовпці від HTML-тегів
            for col in df.columns:
                if df[col].dtype == 'object':
                    df[col] = df[col].map(lambda x: clean_html_tags(str(x)))
        
            # Додаємо стовбчик з індексом регіона
            df['area'] = i
            
            # Додаємо DataFrame до списку
            df_list.append(df)
    
    # Об'єднуємо всі DataFrame в один
    combined_df = pd.concat(df_list, ignore_index=True)
    
    return combined_df


In [237]:
def change_index(df):
    # Створимо словник відповідностей
    replacement_dict = {1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 12: 12, 13: 10, 14: 11, 15: 12, 16: 13, 17: 14, 18: 15, 19: 16, 20: 20, 21: 17, 22: 18, 23: 6, 24: 1, 25: 2, 26: 7, 27: 5}
    
    # Використовуємо заміну 
    df["area"] = df["area"].replace(replacement_dict)
    
    return df


In [238]:
def type_to_int(df):
    df['Year'] = pd.to_numeric(dataframe['Year'], errors='coerce')
    df.dropna(subset=['Year'], inplace=True)
    df['Year'] = dataframe['Year'].astype(int)
    return df

In [239]:
def get_vhi_for_area(dataframe, area, year):
    result = dataframe[(dataframe['area'] == area) & (dataframe['Year'] == year)]['VHI']
    result.to_numpy()
    return result


In [240]:
def get_extremes_mean_median(dataframe, areas, years):
    # Переконайтеся, що areas та years є списками
    if not isinstance(areas, list):
        areas = [areas]
    if not isinstance(years, list):
        years = [years]
    
    # Фільтрація даних для вказаних областей та років
    filtered_df = dataframe[dataframe['area'].isin(areas) & dataframe['Year'].isin(years)]
    
    if filtered_df.empty:
        print("Немає даних для зазначених областей та років.")
        return None

    # Обчислення екстремумів, середнього значення та медіани
    min_vhi = filtered_df['VHI'].min()
    max_vhi = filtered_df['VHI'].max()
    mean_vhi = filtered_df['VHI'].mean()
    median_vhi = filtered_df['VHI'].median()

    results = {
        'min': min_vhi,
        'max': max_vhi,
        'mean': mean_vhi,
        'median': median_vhi,
    }

    return results

In [241]:
def get_vhi_for_area_range(dataframe, areas, start_year, end_year):
    # Переконайтеся, що areas є списком
    if not isinstance(areas, list):
        areas = [areas]
    
    # Фільтрація даних для вказаних областей та діапазону років
    filtered_df = dataframe[
        (dataframe['area'].isin(areas)) &
        (dataframe['Year'].astype(int).between(start_year, end_year))
    ]
    
    if filtered_df.empty:
        print("Немає даних для зазначених областей та діапазону років.")
        return None

    return filtered_df[['Year', 'area', 'VHI']]

In [242]:
def detect_droughts(dataframe):
    drought_threshold=15
    drought_percentage=20
    # Фільтруємо дані, щоб отримати записи з VHI <= 15
    df_drought = dataframe[(dataframe.VHI <= drought_threshold) & (dataframe.VHI != -1)]

    # Знаходимо кількість областей у DataFrame
    total_areas = dataframe['area'].nunique()

    # Обчислюємо кількість областей, які підпадають під екстремальні посухи за роками
    drought_counts = df_drought.groupby('Year')['area'].nunique()

    # Знаходимо роки, де кількість областей з посухами перевищує 20% 
    significant_drought_years = drought_counts[drought_counts > (total_areas * drought_percentage / 100)]

    # Отримаємо дані для повернення
    results = []

    for year in significant_drought_years.index:
        drought_data = df_drought[df_drought['Year'] == year]
        for index, row in drought_data.iterrows():
            results.append({'Year': year, 'Area': row['area'], 'VHI': row['VHI']})

    # Створюємо DataFrame з результатами
    return pd.DataFrame(results)

In [243]:
# Завантаження файлів у католог vhi_data 
download_data(27, save_dir="vhi_data")

# Об'єднуємо файли в один датафрейм
directory_path = 'vhi_data' 
dataframe = load_files_to_dataframe(directory_path)
# Виводими перші 5 рядків отриманого датафрейму
print('Перші 5 рядків отриманого датафрейму')
print(dataframe.head())
print('---------------------------------------------')

# Змінюємо індекси регіонів
dataframe = change_index(dataframe)
print('Перші 5 рядків отриманого датафрейму з новими індексами')
print(dataframe.head())
print('---------------------------------------------')

# Для реалізування наступних процедур змінюємо тип даних на int
dataframe = type_to_int(dataframe)

# Ряд VHI для області за вказаний рік
print('Ряд VHI для області за вказаний рік')
print(get_vhi_for_area(dataframe, area=22, year=1982))
print('---------------------------------------------')

# Пошук min та max, середнього, медіани для вказаних регіонів та років
extremes = get_extremes_mean_median(dataframe, [22,18], [1982])
if extremes:
    print("Results:")
    print(f"min: {extremes['min']}")
    print(f"max: {extremes['max']}")
    print(f"average: {extremes['mean']}")
    print(f"median: {extremes['median']}")
    print('---------------------------------------------')

# Ряд VHI за вказаний діапазон років для вказаних регіонів
print('Ряд VHI за вказаний діапазон років для вказаних регіонів')
print(get_vhi_for_area_range(dataframe, 22, 1982, 1986))
print('---------------------------------------------')

# Роки, назви регіонів з екстремальними посухами та значення VHI
print('Роки, назви регіонів з екстремальними посухами та значення VHI')
print(detect_droughts(dataframe))



Перші 5 рядків отриманого датафрейму
   Year  Week    SMN     SMT    VCI    TCI    VHI  empty  area
0  1982   1.0  0.059  258.24  51.11  48.78  49.95    NaN     1
1  1982   2.0  0.063  261.53  55.89  38.20  47.04    NaN     1
2  1982   3.0  0.063  263.45  57.30  32.69  44.99    NaN     1
3  1982   4.0  0.061  265.10  53.96  28.62  41.29    NaN     1
4  1982   5.0  0.058  266.42  46.87  28.57  37.72    NaN     1
---------------------------------------------
Перші 5 рядків отриманого датафрейму з новими індексами
   Year  Week    SMN     SMT    VCI    TCI    VHI  empty  area
0  1982   1.0  0.059  258.24  51.11  48.78  49.95    NaN    22
1  1982   2.0  0.063  261.53  55.89  38.20  47.04    NaN    22
2  1982   3.0  0.063  263.45  57.30  32.69  44.99    NaN    22
3  1982   4.0  0.061  265.10  53.96  28.62  41.29    NaN    22
4  1982   5.0  0.058  266.42  46.87  28.57  37.72    NaN    22
---------------------------------------------
Ряд VHI для області за вказаний рік
0     49.95
1     47.04